In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from sklearn.semi_supervised import SelfTrainingClassifier
from joblib import load
from keras.models import load_model, clone_model
from sklearn.metrics import f1_score, accuracy_score, precision_score

from copy import deepcopy
import matplotlib.pyplot as plt

import sys
#sys.path.append('C:/Users/JoséEmmanuelParedesR/OneDrive - Exitus Credit/Documentos/SCRIPTS/Scripts_Python')
from Funciones_tablas_validacion import *

In [21]:
########
import os
os.chdir("C:/Users/RobertoRosasGuerrero/Analítica Luisito Valdez/Self Learning/202220809 SELF LEARNING_Nacho/Prueba_Self_Learning/Score_iBillete_Sol20")
#obtener lista de variables por segmento
seg_lst = ['S1','S2','S3','S4']
variables = {}
for seg in seg_lst:
    df_var = pd.read_excel('Variables.xlsx',sheet_name=seg)
    variables = {**variables,**{seg: list(df_var['Variables'].values)}}
    print(seg, len(variables[seg]), 'variables')


S1 31 variables
S2 35 variables
S3 36 variables
S4 28 variables


In [33]:
#pd.set_option("display.Max_rows",None)
{**variables,**{seg: list(df_var['Variables'].values)}}
#{**variables,**{seg: list(df_var['Variables'].values)}}

{'S1': ['AYC_011_Sum_woe',
  'nombreOtorgante_12_AYC_026_Max_woe',
  'tipoContrato_4_AYC_026_Mean_woe',
  'tipoCuenta_3_END_002_Max_woe',
  'EXP_004_Mean_woe',
  'RES_016_BUR_woe',
  'nombreOtorgante_10_SDO_001_BUR_Max_woe',
  'tipoCuenta_3_SDO_002_BUR_Min_woe',
  'INT_001_MOR_Sum_woe',
  'nombreOtorgante_12_INT_017_MOR_Sum_woe',
  'nombreOtorgante_5_INT_049_MOR_Sum_woe',
  'nombreOtorgante_8_INT_034_MOR_Mean_woe',
  'nombreOtorgante_8_INT_065_MOR_Mean_woe',
  'nombreOtorgante_8_INT_097_MOR_Mean_woe',
  'nombreOtorgante_10_INT_053_MOR_Min_woe',
  'nombreOtorgante_10_INT_052_MOR_Max_woe',
  'nombreOtorgante_8_INT_055_MOR_Max_woe',
  'tipoCuenta_1_INT_026_MOR_Sum_woe',
  'tipoCuenta_1_INT_060_MOR_Sum_woe',
  'tipoCuenta_2_INT_086_MOR_Mean_woe',
  'tipoCuenta_1_INT_053_MOR_Min_woe',
  'tipoContrato_1_INT_006_MOR_Sum_woe',
  'tipoContrato_4_INT_063_MOR_Sum_woe',
  'tipoContrato_1_INT_010_MOR_Mean_woe',
  'tipoContrato_4_INT_015_MOR_Mean_woe',
  'tipoContrato_4_INT_057_MOR_Mean_woe',
  'tip

In [28]:
#se cargan los modelos actuales #Score_iBillete_Sol20/
os.chdir("C:/Users/RobertoRosasGuerrero/Analítica Luisito Valdez/Self Learning")
model_S1 = load('Modelos/MLP_S1.pkl')
model_S2 = load_model('Modelos/RNN_S2.h5')
model_S3 = load('Modelos/MLP_S3.pkl')
model_S4 = load('Modelos/MLP_S4.pkl')

In [29]:
#definir variable target
target = 'BGI'

#cargar bases de entrenamiento, etiquetadas y sin etiquetar
# S1
df_e_S1 = pd.read_csv('Bases_seg_Trainning/Base_Entrenamiento_S1.csv')
df_se_S1 = pd.read_csv('Bases_seg_Trainning/Base_Entrenamiento_sinEtiqueta_S1.csv')
df_se_S1[target] = -1 #indicacion de no etiquetado
# S2
df_e_S2 = pd.read_csv('Bases_seg_Trainning/Base_Entrenamiento_S2.csv')
df_se_S2 = pd.read_csv('Bases_seg_Trainning/Base_Entrenamiento_sinEtiqueta_S2.csv')
df_se_S2[target] = -1
# S3
df_e_S3 = pd.read_csv('Bases_seg_Trainning/Base_Entrenamiento_S3.csv')
df_se_S3 = pd.read_csv('Bases_seg_Trainning/Base_Entrenamiento_sinEtiqueta_S3.csv')
df_se_S3[target] = -1
# S4
df_e_S4 = pd.read_csv('Bases_seg_Trainning/Base_Entrenamiento_S4.csv')
df_se_S4 = pd.read_csv('Bases_seg_Trainning/Base_Entrenamiento_sinEtiqueta_S4.csv')
df_se_S4[target] = -1

In [ ]:
#cargar base de comprobación (test)
#S1
df_t_S1 = pd.read_csv('Bases_seg_Trainning/Base_Comprobacion_S1.csv')
#S2
df_t_S2 = pd.read_csv('Bases_seg_Trainning/Base_Comprobacion_S2.csv')
#S3
df_t_S3 = pd.read_csv('Bases_seg_Trainning/Base_Comprobacion_S3.csv')
#S4
df_t_S4 = pd.read_csv('Bases_seg_Trainning/Base_Comprobacion_S4.csv')

In [ ]:
#definir el conjunto de train, test y sin etiquetar
#S1
X_train_S1, y_train_S1 = df_e_S1[variables['S1']], df_e_S1[target]
X_se_S1, y_se_S1 = df_se_S1[variables['S1']], df_se_S1[target]
X_test_S1, y_test_S1 = df_t_S1[variables['S1']], df_t_S1[target]
#S2
X_train_S2, y_train_S2 = df_e_S2[variables['S2']], df_e_S2[target]
X_se_S2, y_se_S2 = df_se_S2[variables['S2']], df_se_S2[target]
X_test_S2, y_test_S2 = df_t_S2[variables['S2']], df_t_S2[target]
#S3
X_train_S3, y_train_S3 = df_e_S3[variables['S3']], df_e_S3[target]
X_se_S3, y_se_S3 = df_se_S3[variables['S3']], df_se_S3[target]
X_test_S3, y_test_S3 = df_t_S3[variables['S3']], df_t_S3[target]
#S4
X_train_S4, y_train_S4 = df_e_S4[variables['S4']], df_e_S4[target]
X_se_S4, y_se_S4 = df_se_S4[variables['S4']], df_se_S4[target]
X_test_S4, y_test_S4 = df_t_S4[variables['S4']], df_t_S4[target]

In [ ]:
# umbral = 0.7
# scores = [] #[f1_score(y_test_S2,model_S2.predict(X_test_S2))]
# num_sin_etiqueta = len(y_se_S2)
# pseudo_etiquetas = [1]
# iteracion = 1

# while pseudo_etiquetas[-1] > 0:
#     if iteracion == 1:
#         pseudo_etiquetas.pop(0)
        
#     #calcular el score del modelo
#     score = f1_score(y_test_S2, predict_clases(model_S2,X_test_S2))
#     scores.append(score)
#     #calcular la probabilidad de las no etiquetadas
#     y_proba_S2 = model_S2.predict(X_se_S2)
#     #obtener la etiqueta (clase) en base a la probabilidad, si ambas probas están debajo del umbral, se asigna -1
#     y_etiquedado_S2 = pd.Series(retornar_clase_y(y_proba_S2, umbral=umbral))
#     #determinar los indices de las que se etiquetaron y se cuentan
#     index_etiquetados = y_etiquedado_S2[y_etiquedado_S2.isin([0,1])].index
#     pseudo_etiquetas.append(len(index_etiquetados)) #contar cuantas se etiquetaron
#     print(f'Iteración #{iteracion}\t Etiquetados: {pseudo_etiquetas[-1]}. Etiquetado total: {sum(pseudo_etiquetas)}/{num_sin_etiqueta}')
#     #agregar los etiquetados al conjunto de entrenamiento
#     X_train_S2 = pd.concat([X_train_S2, X_se_S2.loc[index_etiquetados]]).reset_index(drop=True)
#     y_train_S2 = pd.concat([y_train_S2,y_etiquedado_S2.loc[index_etiquetados]]).reset_index(drop=True)
#     #eliminar los etiquetados del conjunto de no etiquetados
#     X_se_S2 = X_se_S2.drop(index=index_etiquetados).reset_index(drop=True)
#     y_se_S2 = y_se_S2.drop(index=index_etiquetados).reset_index(drop=True)
#     #se reentrena
#     model_S2.fit(X_train_S2, y_train_S2)
#     #agregar al conteo de la iteracion
#     iteracion += 1
    

In [ ]:
def reentrenar_modelo(modelo, c_train, c_test, c_se, umbral, n_iter = None):
    
    #hacer una copia profunda de las variables para que no se modifiquen las originales
    model = deepcopy(modelo)
    X_train = c_train[0].copy()
    y_train = c_train[1].copy()
    X_test = c_test[0].copy()
    y_test = c_test[1].copy()
    X_se = c_se[0].copy()
    y_se = c_se[1].copy()
    
    #bandera para saber el valor del umbral
    print('Umbral: ', umbral)
    #comienza el proceso de reentrenamiento
    scores = [] #lista para guardar el score
    num_sin_etiqueta = len(y_se)
    pseudo_etiquetas = [1]
    iteracion = 1

    while pseudo_etiquetas[-1] > 0:
        if iteracion == 1:
            pseudo_etiquetas.pop(0)
            
        #calcular el score del modelo
        score = f1_score(y_test, model.predict(X_test))
        scores.append(score)
        #calcular la probabilidad de las no etiquetadas
        y_proba = model.predict_proba(X_se)
        #obtener la etiqueta (clase) en base a la probabilidad, si ambas probas están debajo del umbral, se asigna -1
        y_etiquedado = pd.Series(retornar_clase_y(y_proba, umbral))
        #determinar los indices de las que se etiquetaron y se cuentan
        index_etiquetados = y_etiquedado[y_etiquedado.isin([0,1])].index
        pseudo_etiquetas.append(len(index_etiquetados)) #contar cuantas se etiquetaron
        print(f'Iteración #{iteracion}\t Etiquetados: {pseudo_etiquetas[-1]}. Etiquetado total: {sum(pseudo_etiquetas)}/{num_sin_etiqueta}')
        #agregar los etiquetados al conjunto de entrenamiento
        X_train = pd.concat([X_train, X_se.loc[index_etiquetados]]).reset_index(drop=True)
        y_train = pd.concat([y_train,y_etiquedado.loc[index_etiquetados]]).reset_index(drop=True)
        #eliminar los etiquetados del conjunto de no etiquetados
        X_se = X_se.drop(index=index_etiquetados).reset_index(drop=True)
        y_se = y_se.drop(index=index_etiquetados).reset_index(drop=True)
        #se reentrena
        model.fit(X_train, y_train)
        #agregar al conteo de la iteracion
        iteracion += 1
        #cortar el proceso si en la primera vuelta se etiqueta a todos
        if X_se.shape[0] == 0:
            break
        #cortar por número de iteraciones
        if n_iter == None:
            pass
        else:
            if iteracion == n_iter + 1:
                break
        
    return model, scores, pseudo_etiquetas

def reentrenar_modelo_RN(modelo, c_train, c_test, c_se, umbral, n_iter=None):
    
    #hacer una copia profunda de las variables para que no se modifiquen las originales
    model = clone_model(modelo)
    X_train = c_train[0].copy()
    y_train = c_train[1].copy()
    X_test = c_test[0].copy()
    y_test = c_test[1].copy()
    X_se = c_se[0].copy()
    y_se = c_se[1].copy()
    
    #bandera para saber el valor del umbral
    print('Umbral: ', f'{umbral:.2f}')
    #comienza el proceso de reentrenamiento
    scores = [] #lista para guardar el score
    num_sin_etiqueta = len(y_se)
    pseudo_etiquetas = [1]
    iteracion = 1

    while pseudo_etiquetas[-1] > 0:
        if iteracion == 1:
            pseudo_etiquetas.pop(0)
            
        #calcular el score del modelo
        score = f1_score(y_test, predict_clases(model,X_test))
        scores.append(score)
        #calcular la probabilidad de las no etiquetadas
        y_proba = model.predict(X_se)
        #obtener la etiqueta (clase) en base a la probabilidad, si ambas probas están debajo del umbral, se asigna -1
        y_etiquedado = pd.Series(retornar_clase_y(y_proba, umbral))
        #determinar los indices de las que se etiquetaron y se cuentan
        index_etiquetados = y_etiquedado[y_etiquedado.isin([0,1])].index
        pseudo_etiquetas.append(len(index_etiquetados)) #contar cuantas se etiquetaron
        print(f'Iteración #{iteracion}\t Etiquetados: {pseudo_etiquetas[-1]}. Etiquetado total: {sum(pseudo_etiquetas)}/{num_sin_etiqueta}')
        #agregar los etiquetados al conjunto de entrenamiento
        X_train = pd.concat([X_train, X_se.loc[index_etiquetados]]).reset_index(drop=True)
        y_train = pd.concat([y_train,y_etiquedado.loc[index_etiquetados]]).reset_index(drop=True)
        #eliminar los etiquetados del conjunto de no etiquetados
        X_se = X_se.drop(index=index_etiquetados).reset_index(drop=True)
        y_se = y_se.drop(index=index_etiquetados).reset_index(drop=True)
        #compilar el modelo
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #se reentrena
        model.fit(X_train, y_train, verbose=0)
        #agregar al conteo de la iteracion
        iteracion += 1
        #cortar el proceso si en la primera vuelta se etiqueta a todos
        if X_se.shape[0] == 0:
            break
        #cortar por número de iteraciones
        if n_iter == None:
            pass
        else:
            if iteracion == n_iter + 1:
                break
        
    return model, scores, pseudo_etiquetas

## Determinar los mejores valores de umbral y n_iteraciones para cada segmento

In [ ]:
umbral_valores = np.arange(0.50, 1.05, 0.05)
umbral_valores = np.append(umbral_valores, 0.99)
umbral_valores = sorted(umbral_valores)
umbral_valores

In [ ]:
# umbral = 0.7
# scores = [] #[f1_score(y_test_S1,model_S1.predict(X_test_S1))]
# num_sin_etiqueta = len(y_se_S1)
# pseudo_etiquetas = [1]
# iteracion = 1

# while pseudo_etiquetas[-1] > 0:
#     if iteracion == 1:
#         pseudo_etiquetas.pop(0)
        
#     #calcular el score del modelo
#     score = f1_score(y_test_S1, model_S1.predict(X_test_S1))
#     scores.append(score)
#     #calcular la probabilidad de las no etiquetadas
#     y_proba_S1 = model_S1.predict_proba(X_se_S1)
#     #obtener la etiqueta (clase) en base a la probabilidad, si ambas probas están debajo del umbral, se asigna -1
#     y_etiquedado_S1 = pd.Series(retornar_clase_y(y_proba_S1, umbral=umbral))
#     #determinar los indices de las que se etiquetaron y se cuentan
#     index_etiquetados = y_etiquedado_S1[y_etiquedado_S1.isin(model_S1.classes_)].index
#     pseudo_etiquetas.append(len(index_etiquetados)) #contar cuantas se etiquetaron
#     print(f'Iteración #{iteracion}\t Etiquetados: {pseudo_etiquetas[-1]}. Etiquetado total: {sum(pseudo_etiquetas)}/{num_sin_etiqueta}')
#     #agregar los etiquetados al conjunto de entrenamiento
#     X_train_S1 = pd.concat([X_train_S1, X_se_S1.loc[index_etiquetados]]).reset_index(drop=True)
#     y_train_S1 = pd.concat([y_train_S1,y_etiquedado_S1.loc[index_etiquetados]]).reset_index(drop=True)
#     #eliminar los etiquetados del conjunto de no etiquetados
#     X_se_S1 = X_se_S1.drop(index=index_etiquetados).reset_index(drop=True)
#     y_se_S1 = y_se_S1.drop(index=index_etiquetados).reset_index(drop=True)
#     #se reentrena
#     model_S1.fit(X_train_S1, y_train_S1)
#     #agregar al conteo de la iteracion
#     iteracion += 1
    

In [ ]:
umbrales = []
scores_umbral = []
pseudo_etiquetas_umbral = []
   
for umbral in umbral_valores:
    umbrales.append(umbral)
    scores, pseudo_etiquetas = reentrenar_modelo(model_S1, (X_train_S1, y_train_S1), (X_test_S1,y_test_S1), (X_se_S1, y_se_S1), umbral=umbral)[1:]
    scores_umbral.append(scores)
    pseudo_etiquetas_umbral.append(pseudo_etiquetas)

In [ ]:
plt.figure(1, figsize=(10,7))
plt.subplot(2,1,1)
for umbral, scores in zip(umbrales, scores_umbral):
    plt.plot(range(len(scores)), scores, label = f'{umbral:.2f}')
plt.xlabel('Iteración')
plt.ylabel('f1 score')
plt.title('Performance autoentrenamiento modelo S1')
plt.legend(bbox_to_anchor=(1,1))

plt.subplot(2,1,2)
for umbral, pseudo_etiquetas in zip(umbrales,pseudo_etiquetas_umbral):
    pseudo_etiquetas_total = [sum(pseudo_etiquetas[:i]) for i in range(1,len(pseudo_etiquetas)+1)]
    plt.plot(range(len(pseudo_etiquetas_total)), pseudo_etiquetas_total, label = f'{umbral:.2f}')
plt.xlabel('Iteración')
plt.ylabel('Total etiquetados')
#plt.title('Etiquetado autoentrenamiento modelo S1')
#plt.legend(bbox_to_anchor=(1,1))
plt.show()

In [ ]:
umbrales = []
scores_umbral = []
pseudo_etiquetas_umbral = []
   
for umbral in umbral_valores:
    umbrales.append(umbral)
    scores, pseudo_etiquetas = reentrenar_modelo_RN(model_S2, (X_train_S2, y_train_S2), (X_test_S2,y_test_S2), (X_se_S2, y_se_S2), umbral=umbral)[1:]
    scores_umbral.append(scores)
    pseudo_etiquetas_umbral.append(pseudo_etiquetas)

In [ ]:
plt.figure(3, figsize=(10,7))
plt.subplot(2,1,1)
for umbral, scores in zip(umbrales, scores_umbral):
    plt.plot(range(len(scores)), scores, label = f'{umbral:.2f}')
plt.xlabel('Iteración')
plt.ylabel('f1 score')
plt.title('Performance autoentrenamiento modelo S2')
plt.legend(bbox_to_anchor=(1,1))

plt.subplot(2,1,2)
for umbral, pseudo_etiquetas in zip(umbrales,pseudo_etiquetas_umbral):
    pseudo_etiquetas_total = [sum(pseudo_etiquetas[:i]) for i in range(1,len(pseudo_etiquetas)+1)]
    plt.plot(range(len(pseudo_etiquetas_total)), pseudo_etiquetas_total, label = f'{umbral:.2f}')
plt.xlabel('Iteración')
plt.ylabel('Total etiquetados')

plt.show()

In [ ]:
umbrales = []
scores_umbral = []
pseudo_etiquetas_umbral = []
   
for umbral in umbral_valores:
    umbrales.append(umbral)
    scores, pseudo_etiquetas = reentrenar_modelo(model_S3, (X_train_S3, y_train_S3), (X_test_S3,y_test_S3), (X_se_S3, y_se_S3), umbral=umbral)[1:]
    scores_umbral.append(scores)
    pseudo_etiquetas_umbral.append(pseudo_etiquetas)

In [ ]:
plt.figure(3, figsize=(10,7))
plt.subplot(2,1,1)
for umbral, scores in zip(umbrales, scores_umbral):
    plt.plot(range(len(scores)), scores, label = f'{umbral:.2f}')
plt.xlabel('Iteración')
plt.ylabel('f1 score')
plt.title('Performance autoentrenamiento modelo S3')
plt.legend(bbox_to_anchor=(1,1))

plt.subplot(2,1,2)
for umbral, pseudo_etiquetas in zip(umbrales,pseudo_etiquetas_umbral):
    pseudo_etiquetas_total = [sum(pseudo_etiquetas[:i]) for i in range(1,len(pseudo_etiquetas)+1)]
    plt.plot(range(len(pseudo_etiquetas_total)), pseudo_etiquetas_total, label = f'{umbral:.2f}')
plt.xlabel('Iteración')
plt.ylabel('Total etiquetados')

plt.show()

In [ ]:
umbrales = []
scores_umbral = []
pseudo_etiquetas_umbral = []
   
for umbral in umbral_valores:
    umbrales.append(umbral)
    scores, pseudo_etiquetas = reentrenar_modelo(model_S4, (X_train_S4, y_train_S4), (X_test_S4,y_test_S4), (X_se_S4, y_se_S4), umbral=umbral)[1:]
    scores_umbral.append(scores)
    pseudo_etiquetas_umbral.append(pseudo_etiquetas)

In [ ]:
plt.figure(4, figsize=(10,7))
plt.subplot(2,1,1)
for umbral, scores in zip(umbrales, scores_umbral):
    plt.plot(range(len(scores)), scores, label = f'{umbral:.2f}')
plt.xlabel('Iteración')
plt.ylabel('f1 score')
plt.title('Performance autoentrenamiento modelo S4')
plt.legend(bbox_to_anchor=(1,1))

plt.subplot(2,1,2)
for umbral, pseudo_etiquetas in zip(umbrales,pseudo_etiquetas_umbral):
    pseudo_etiquetas_total = [sum(pseudo_etiquetas[:i]) for i in range(1,len(pseudo_etiquetas)+1)]
    plt.plot(range(len(pseudo_etiquetas_total)), pseudo_etiquetas_total, label = f'{umbral:.2f}')
plt.xlabel('Iteración')
plt.ylabel('Total etiquetados')

plt.show()

## Estudio de las tablas de validación

In [ ]:
def obtener_score(modelo, base, variables):
    base_output = base.copy()
    X = base_output[variables]
    #calcular las proba de las predicciones
    preds = modelo.predict_proba(X) #### Solo se califica la base sin smote
    preds = np.array([ preds[i][0] for i in range(len(preds))])
    base_output['score1'] = np.around(preds*1000, decimals = 0)
    return base_output

In [ ]:
#numero de bandas
N=5

#cargar bases para calificar y generar las tablas de validacion
base_S1 = pd.read_csv('Bases_seg_Trainning/Base_Calificar_S1.csv')
base_S2 = pd.read_csv('Bases_seg_Trainning/Base_Calificar_S2.csv')
base_S3 = pd.read_csv('Bases_seg_Trainning/Base_Calificar_S3.csv')
base_S4 = pd.read_csv('Bases_seg_Trainning/Base_Calificar_S4.csv')

In [ ]:
st_model_S1 = reentrenar_modelo(model_S1, (X_train_S1, y_train_S1), (X_test_S1,y_test_S1), (X_se_S1, y_se_S1), umbral=0.8, n_iter=2)[0]

In [ ]:
tv_S1 = tablas_de_entrenamiento_y_validacion(obtener_score(model_S1, base_S1, variables['S1']), N,  string_score = 'score1', string_bgi = 'BGI')
print('train')
display(tv_S1[0])
print('test')
display(tv_S1[1])
print('Calificacion del modelo')
cal_modelo_S1 = calificar_modelo(tv_S1[0], tv_S1[1], dist_malos=0.057, long_bandas=93, KS_min=0.265,dif_KS=0.05,dist_odds=0.3)
display(cal_modelo_S1)

In [ ]:
tv_st_S1 = tablas_de_entrenamiento_y_validacion(obtener_score(st_model_S1, base_S1, variables['S1']), N,  string_score = 'score1', string_bgi = 'BGI')
print('train')
display(tv_st_S1[0])
print('test')
display(tv_st_S1[1])
print('Calificacion del modelo')
cal_modelo_S1 = calificar_modelo(tv_st_S1[0], tv_st_S1[1], dist_malos=0.057, long_bandas=93, KS_min=0.265,dif_KS=0.05,dist_odds=0.3)
display(cal_modelo_S1)

In [ ]:
calificar_modelo(tv_st_S1[0], tv_st_S1[1], dist_malos=0.09, long_bandas=120, KS_min=0.265,dif_KS=0.05,dist_odds=0.19)

In [ ]:
?calificar_modelo